In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
mongo_client = MongoClient('18.228.116.164', 27017)
mongo_db = mongo_client['inatel']
mongo_collection = mongo_db['in242']

In [3]:
dados = list(mongo_collection.find())
df = pd.DataFrame(dados)

In [4]:
df.head()

,_id,temperatura,data_coleta
0,5db7839b84a2ef98b84f7ef5,17.748594,2019-10-28 21:11:07.955
1,5db7839c84a2ef98b84f7ef6,27.191536,2019-10-28 21:11:08.956
2,5db7839d84a2ef98b84f7ef7,17.749806,2019-10-28 21:11:09.958
3,5db7839e84a2ef98b84f7ef8,22.026900,2019-10-28 21:11:10.959
4,5db7839f84a2ef98b84f7ef9,19.379516,2019-10-28 21:11:11.960


In [5]:
df.count()

_id            103218
temperatura    103218
data_coleta    103218
dtype: int64